In [0]:
# ============================================================================
# Camada Gold - Análise Mercado Cartas Executivo - Magic: The Gathering
# Pipeline 100% PySpark DataFrame API  
# ============================================================================

# ============================================================================
# BIBLIOTECAS UTILIZADAS
# ============================================================================
import logging
from datetime import datetime
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *
from pyspark.sql.types import *

# ============================================================================
# IMPORTAÇÃO DO MÓDULO UTILITÁRIO
# ============================================================================
%run ./gold_utils

# ============================================================================
# CONFIGURAÇÃO COM MÓDULO UTILITÁRIO
# ============================================================================
TABLE_NAME = "TB_ANALISE_MERCADO_CARTAS_EXECUTIVO"

# Inicializar processador com configuração manual 
def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Segredo '{secret_name}' não configurado")

config = create_manual_config(
    catalog_name="magic_the_gathering", 
    s3_bucket=get_secret("s3_bucket"),  
    s3_gold_prefix="magic_the_gathering/gold"
)
processor = GoldTableProcessor(TABLE_NAME, config)
spark = processor.spark



In [0]:
# ============================================================================
# CARREGAMENTO DE DADOS SILVER COM MÓDULO
# ============================================================================
dfs = processor.load_silver_data(['cards', 'prices'])
df_cards = dfs['cards']
df_prices = dfs['prices']

# ============================================================================
# JOIN PRINCIPAL E RESOLUÇÃO DE AMBIGUIDADES
# ============================================================================
df = df_cards.join(df_prices, ["NME_CARD", "COD_SET"], "inner")

In [0]:
# Usar make_date com RELEASE_YEAR e RELEASE_MONTH para criar DATA_REF de negócio
df = df.withColumn("DATA_REF", make_date(col("cards.RELEASE_YEAR"), col("cards.RELEASE_MONTH"), lit(1)))

# ============================================================================
# AGREGAÇÕES DE MERCADO EXECUTIVO
# ============================================================================

# 1. Análise por Segmento de Mercado
# Incluir NME_CARD no groupBy para permitir JOIN posterior
df_gold = (
    df.groupBy("DATA_REF", "NME_SET", "NME_CARD_TYPE", "cards.NME_RARITY", "NME_ARTIST", "NME_CARD")
      .agg(
          round(sum("VLR_USD").cast("float"), 2).alias("VALOR_TOTAL_MERCADO"),
          countDistinct("NME_CARD").cast("int").alias("QTD_CARTAS_ATIVAS"),
          round(avg("VLR_USD").cast("float"), 2).alias("VALOR_MEDIO_CARTA"),
          round(expr("percentile_approx(VLR_USD, 0.5)").cast("float"), 2).alias("TICKET_MEDIANA")
      )
      .withColumnRenamed("cards.NME_RARITY", "NME_RARITY")  # Rename para remover prefixo
)

# 2. Market Share por Set
window_set = Window.partitionBy("DATA_REF")
df_gold = df_gold.withColumn(
    "MARKET_SHARE_SET",
    round((col("VALOR_TOTAL_MERCADO") / sum("VALOR_TOTAL_MERCADO").over(window_set)).cast("float"), 4)
)

# 3. Análise de Valorização por Carta
window_card = Window.partitionBy("NME_CARD").orderBy("DATA_REF")
df_val = (
    df.groupBy("DATA_REF", "NME_CARD")
      .agg(round(sum("VLR_USD").cast("float"), 2).alias("VALOR_ATUAL"))
      .withColumn("VALOR_ANTERIOR", lag("VALOR_ATUAL").over(window_card))
      .withColumn("VARIACAO_PERC_VALOR", 
                  when(col("VALOR_ANTERIOR").isNotNull() & (col("VALOR_ANTERIOR") != 0),
                       round(((col("VALOR_ATUAL") - col("VALOR_ANTERIOR")) / col("VALOR_ANTERIOR")).cast("float"), 4))
                  .otherwise(lit(None)))
)

# 4. Rankings de Performance
# Usar partição por DATA_REF para otimizar performance e evitar warning
window_risco = Window.partitionBy("DATA_REF").orderBy(col("VARIACAO_PERC_VALOR").desc())
df_val = df_val.withColumn("RANK_VALORIZACAO", row_number().over(window_risco).cast("int"))

In [0]:
# ============================================================================
# CONSOLIDAÇÃO FINAL
# ============================================================================

# Join com dados de valorização
# Corrigir JOIN para incluir NME_CARD e evitar multiplicação de linhas
df_final = df_gold.join(
    df_val.select("DATA_REF", "NME_CARD", "VARIACAO_PERC_VALOR", "RANK_VALORIZACAO"),
    ["DATA_REF", "NME_CARD"], "left"
)

# Adicionar métricas de análise executiva
# Categorizar por valor de mercado (mais adequado para análise executiva)
df_final = df_final.withColumn("CATEGORIA_VALOR_MERCADO",
    when(col("VALOR_TOTAL_MERCADO") > 1000000, "Alto Valor (>1M)")
    .when(col("VALOR_TOTAL_MERCADO") > 100000, "Médio Valor (100K-1M)")
    .when(col("VALOR_TOTAL_MERCADO") > 10000, "Baixo Valor (10K-100K)")
    .otherwise("Baixíssimo Valor (<10K)")
)

# Categorizar por market share
df_final = df_final.withColumn("CATEGORIA_MARKET_SHARE",
    when(col("MARKET_SHARE_SET") > 0.1, "Alto Market Share (>10%)")
    .when(col("MARKET_SHARE_SET") > 0.05, "Medio Market Share (5-10%)")
    .when(col("MARKET_SHARE_SET") > 0.01, "Baixo Market Share (1-5%)")
    .otherwise("Minimo Market Share (<1%)")
)


# ============================================================================
# ESCRITA DA TABELA COM MÓDULO
# ============================================================================
# Partições otimizadas para consultas executivas:
# - DATA_REF: Para análises temporais
# - NME_SET: Para filtrar por sets específicos
# - NME_CARD_TYPE: Para análises por tipo de carta
# - NME_RARITY: Para análises por raridade
processor.save_gold_table(df_final, partition_cols=["DATA_REF", "NME_SET", "NME_CARD_TYPE", "NME_RARITY"])

# ============================================================================
# FINALIZAÇÃO
# ============================================================================
print("TB_ANALISE_MERCADO_CARTAS_EXECUTIVO modularizada criada com sucesso!")